# Chapter 10: Introduction to Artificial Neural Networks

In [1]:
%load_ext autotime

In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os
from datetime import datetime

time: 1.43 s


## Loading MNIST dataset

In [3]:
mnist = fetch_mldata('MNIST original')
mnist

/home/caiolmartinelli/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/home/caiolmartinelli/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


{'DESCR': 'mldata.org dataset: mnist-original',
 'COL_NAMES': ['label', 'data'],
 'target': array([0., 0., 0., ..., 9., 9., 9.]),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

time: 41.8 ms


In [4]:
X, y = mnist['data'], mnist['target']
X.shape

(70000, 784)

time: 167 ms


In [5]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = np.int32(y[:60000]), np.int32(y[60000:])

time: 51.2 ms


In [6]:
scaler = StandardScaler()

time: 67.9 ms


In [7]:
X_train_scaled = scaler.fit_transform(np.float32(X_train))
X_test_scaled = scaler.transform(np.float32(X_test))

time: 1.23 s


## Training an MLP with TensorFlow's High-Level API

In [8]:
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_scaled)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                         feature_columns=feature_cols)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train_scaled, y_train, batch_size=50, steps=40000)
;


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorf

''

time: 1min 57s


In [9]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpw1e82sft/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9459

time: 355 ms


## Training a DNN Using Plain TensorFlow

### Construction Phase

Creating the diagram in two different ways, also creating a initialization node and a Saver.

In [10]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

time: 720 µs


In [11]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

time: 69.7 ms


Creating neuron layers to create the DNN

In [13]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        Z = tf.matmul(X, W) +  b
        if activation is not None:
            return activation(Z)
        else:
            return Z

time: 4.13 ms


In [14]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name='hidden1',
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(X, n_hidden2, name='hidden2',
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name='outputs')

time: 51.7 ms


Using tf.layers to create the DNN

In [15]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1',
                           activation=tf.nn.relu)
    hidden2 = tf.layers.dense(X, n_hidden2, name='hidden2',
                           activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

Instructions for updating:
Use keras.layers.dense instead.
time: 72.5 ms


In [16]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

time: 13.4 ms


In [17]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

time: 58.5 ms


In [19]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

time: 14.2 ms


In [20]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

time: 33.9 ms


### Execution Phase

In [22]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
time: 4.23 s


In [23]:
n_epochs = 40
batch_size = 50

time: 2.75 ms


In [24]:
if not os.path.exists('chp10/models/'):
    if not os.path.exists('chp10/'):
        os.mkdir('chp10')
    os.mkdir('chp10/models')

time: 2.37 ms


In [25]:
if not os.path.exists('chp10/models/dnn'):
    os.mkdir('chp10/models/dnn')

time: 4.16 ms


In [26]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, 
                                           y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
    save_path = saver.save(sess, "chp10/models/dnn/my_model_final.ckpt")

0 Train accuracy: 0.86 Val accuracy: 0.8902
1 Train accuracy: 0.82 Val accuracy: 0.9054
2 Train accuracy: 0.88 Val accuracy: 0.9136
3 Train accuracy: 0.92 Val accuracy: 0.9228
4 Train accuracy: 0.88 Val accuracy: 0.9282
5 Train accuracy: 0.92 Val accuracy: 0.9312
6 Train accuracy: 1.0 Val accuracy: 0.9342
7 Train accuracy: 0.84 Val accuracy: 0.9362
8 Train accuracy: 0.92 Val accuracy: 0.937
9 Train accuracy: 0.94 Val accuracy: 0.939
10 Train accuracy: 1.0 Val accuracy: 0.9436
11 Train accuracy: 0.96 Val accuracy: 0.9474
12 Train accuracy: 0.9 Val accuracy: 0.9478
13 Train accuracy: 0.92 Val accuracy: 0.9492
14 Train accuracy: 0.98 Val accuracy: 0.9512
15 Train accuracy: 0.96 Val accuracy: 0.9526
16 Train accuracy: 0.9 Val accuracy: 0.9538
17 Train accuracy: 0.94 Val accuracy: 0.9542
18 Train accuracy: 0.96 Val accuracy: 0.9566
19 Train accuracy: 0.96 Val accuracy: 0.9574
20 Train accuracy: 0.9 Val accuracy: 0.9594
21 Train accuracy: 0.96 Val accuracy: 0.9586
22 Train accuracy: 0.96 Val